# Загрузка данных и библиотек



In [ ]:
!pip install ultralytics

In [ ]:
!pip install optuna

Если датасет лежит на drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Обучение и настройка модели


Библиотеки


In [5]:
import torch
from ultralytics import YOLO
import optuna
import subprocess
import os

Установка рабочей среды


In [6]:
# Установка переменной среды для отладки CUDA ошибок
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# Проверка доступности GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

Using device: cpu


In [7]:
# Функция для обучения модели и возврата значения метрики
def objective(trial):
    # Настройка гиперпараметров с использованием Optuna
    batch_size = trial.suggest_categorical('batch_size', [16, 32])
    epochs = 150  # Используем большее количество эпох
    img_size = 640
    lr0 = trial.suggest_float('lr0', 1e-5, 1e-2, log=True)
    momentum = trial.suggest_float('momentum', 0.8, 0.95)
    weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-4, log=True)
    num_workers = 16  # Фиксированное количество потоков для загрузки данных

    # Загрузка предобученной модели и перемещение на устройство
    model = YOLO('yolov8m.pt').to(device)

    # Обучение модели с оптимизированными параметрами
    results = model.train(
        data='/home/jupyter/datasphere/project/dataset/dataset.yaml',
        epochs=epochs,
        batch=batch_size,
        imgsz=img_size,
        workers=num_workers,
        lr0=lr0,
        momentum=momentum,
        weight_decay=weight_decay,
        device=device,
        iou=0.3,
        lrf=0.01
    )

    # Возвращаем значение метрики, которую нужно минимизировать (например, значение потерь на валидационном наборе)
    return results.metrics['val']['loss']

Запуск обучения

In [ ]:
# Создание и запуск Optuna study
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

# Вывод лучших параметров и значения метрики
print(f"Best params: {study.best_params}")
print(f"Best score: {study.best_value}")

# Вызов функции мониторинга ресурсов
def monitor_resources():
    # Команда для вывода состояния GPU
    result = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE)
    print(result.stdout.decode())

monitor_resources()

# Предсказание результатов


Настройка девайса

In [ ]:
import torch
import os
import cv2

device = torch.device("cuda:0")

Сбор данных для разметки

In [ ]:
filenames = []
train_dir = '/home/jupyter/datasphere/project/test_dataset'
for i in os.walk(train_dir):
  filenames = i[2]
for index, i in enumerate(filenames):
  filenames[index] = train_dir + '/' + i

Разметка

In [ ]:
from ultralytics import YOLO

model = YOLO("/home/jupyter/datasphere/project/runs/detect/train10/weights/best.pt").to(device)


answer = {'filename': [], 'class_id': [], 'rel_x': [], 'rel_y': [], 'width': [], 'height': []}
length = len(filenames)
for index, i in enumerate(filenames):
  HEIGHT, WIDTH = cv2.imread(i).shape[0:2]
  print(f"{index}/{length}")
  result = model([i])[0]
  boxes = result.boxes.xyxy.tolist()
  classes = result.boxes.cls.tolist()
  names = result.names
  confidences = result.boxes.conf.tolist()

  for box, cls, conf in zip(boxes, classes, confidences):
      x1, y1, x2, y2 = box
      confidence = conf
      detected_class = cls
      name = names[int(cls)]
      answer['filename'].append(i.split('/')[-1])
      answer['class_id'].append(cls)
      answer['rel_x'].append((x1 + ((x2 - x1) / 2)) / WIDTH)
      answer['rel_y'].append((y1 + ((y2 - y1) / 2)) / HEIGHT)
      answer['width'].append((x2 - x1) / WIDTH)
      answer['height'].append((y2 - y1) / HEIGHT)

Формирование результата

In [ ]:
import pandas as pd

df = pd.DataFrame(answer)
df.to_csv("subm.csv", index=False, sep=";")